In [1]:
import pickle
import numpy as np
import torch
import torch.nn as nn
import janestreet

In [2]:
CACHE_PATH = '../input/pytorch-mix-1'
NFOLDS = 5

scalers = pickle.load(open('../input/scalers-for-copy/scalers_10_2.pkl','rb'))
f_notused = pickle.load(open('../input/scalers-for-copy/fnotused_10_2.pkl','rb'))
f_mean = np.load(f'{CACHE_PATH}/f_mean_online.npy')

In [3]:
def save_pickle(dic, save_path):
    with open(save_path, 'wb') as f:
        pickle.dump(dic, f)

def load_pickle(load_path):
    with open(load_path, 'rb') as f:
        message_dict = pickle.load(f)
    return message_dict

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.batch_norm0 = nn.BatchNorm1d(len(scalers))
        self.dropout0 = nn.Dropout(0.2)

        dropout_rate = 0.2
        hidden_size = 256
        self.dense1 = nn.Linear(len(scalers), hidden_size)
        self.batch_norm1 = nn.BatchNorm1d(hidden_size)
        self.dropout1 = nn.Dropout(dropout_rate)

        self.dense2 = nn.Linear(hidden_size+len(scalers), hidden_size)
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(dropout_rate)

        self.dense3 = nn.Linear(hidden_size+hidden_size, hidden_size)
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(dropout_rate)

        self.dense4 = nn.Linear(hidden_size+hidden_size, hidden_size)
        self.batch_norm4 = nn.BatchNorm1d(hidden_size)
        self.dropout4 = nn.Dropout(dropout_rate)

        self.dense5 = nn.Linear(hidden_size+hidden_size, 5)

        self.Relu = nn.ReLU(inplace=True)
        self.PReLU = nn.PReLU()
        self.LeakyReLU = nn.LeakyReLU(negative_slope=0.01, inplace=True)
        self.RReLU = nn.RReLU()

    def forward(self, x):
        x = self.batch_norm0(x)
        x = self.dropout0(x)

        x1 = self.dense1(x)
        x1 = self.batch_norm1(x1)
        x1 = self.LeakyReLU(x1)
        x1 = self.dropout1(x1)

        x = torch.cat([x, x1], 1)

        x2 = self.dense2(x)
        x2 = self.batch_norm2(x2)
        x2 = self.LeakyReLU(x2)
        x2 = self.dropout2(x2)

        x = torch.cat([x1, x2], 1)

        x3 = self.dense3(x)
        x3 = self.batch_norm3(x3)
        x3 = self.LeakyReLU(x3)
        x3 = self.dropout3(x3)

        x = torch.cat([x2, x3], 1)

        x4 = self.dense4(x)
        x4 = self.batch_norm4(x4)
        x4 = self.LeakyReLU(x4)
        x4 = self.dropout4(x4)

        x = torch.cat([x3, x4], 1)

        x = self.dense5(x)

        return x

In [4]:
device = torch.device("cuda:0")

model_list = []
tmp = np.zeros(len(scalers))
for _fold in range(NFOLDS):
    torch.cuda.empty_cache()
    model = Model()
    model.to(device)
    model_weights = f"{CACHE_PATH}/online_model{_fold}.pth"
    model.load_state_dict(torch.load(model_weights))
    model.eval()
    model_list.append(model)

In [5]:
env = janestreet.make_env()
iter_test = env.iter_test()
f = len(scalers)
for (test_df, sample_prediction_df) in iter_test:
    test_df = test_df.drop(columns=f_notused)
    
    if (test_df.weight.all() <= 0):
        sample_prediction_df.action = 0
        env.predict(sample_prediction_df)
    else:
        test_df = test_df.drop(columns=['weight']).values
            
        if np.isnan(test_df.sum()):
            test_df = np.nan_to_num(test_df) + np.isnan(test_df) * f_mean
        
        
        for scaler,i in zip(scalers,range(f)):
            if (scaler!='passthrough'):
                test_df[:,i:i+1] = scaler.transform(test_df[:,i:i+1])
        
        pred = np.zeros((1, 5))
        for model in model_list:
            pred += model(torch.tensor(test_df, dtype=torch.float).to(device)).sigmoid().detach().cpu().numpy() / NFOLDS
        pred = np.median(pred)
        sample_prediction_df.action = np.where(pred >= 0.5, 1, 0).astype(int)
        
        env.predict(sample_prediction_df)